**Rapid-Fire Scenario Exposure** </br></br>
The Rapid-Fire Scenario Exposure module is designed to help users develop an intuition for distinguishing between “fair” and “unfair” outcomes. Using a set of examples from various disciplines, users can quickly assess the overall fairness of the situation. Fifteen examples are selected at random, each prompting the user to make a binary decision of FAIR or UNFAIR. Upon completion, the results of each question are tallied, and users are given an overall score. Users can replay the module as often as they like, and, like all modules, the underlying questions can be augmented and updated over time.

In [ ]:
#@title pip install
!pip install ipywidgets

In [ ]:
#@title Imports
import random
import time
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Layout

In [ ]:
#@title Fairness Pattern Trainer
class FairnessPatternTrainer:
    def __init__(self):
        self.scenarios = [
            # UNFAIR scenarios
            {
                "text": "Hiring algorithm: 80% accuracy for men, 60% accuracy for women, same qualifications",
                "is_unfair": True,
                "explanation": "Different accuracy rates across gender groups indicates systematic unfairness"
            },
            {
                "text": "Loan approval: 70% approval rate for zip code 10001, 30% for zip code 10002, similar income levels",
                "is_unfair": True,
                "explanation": "Geographic disparities with similar income suggest unfair redlining practices"
            },
            {
                "text": "Medical AI: Diagnostic accuracy 85% for light skin, 65% for dark skin",
                "is_unfair": True,
                "explanation": "Systematic accuracy differences by race indicate unfair training data representation"
            },
            {
                "text": "Resume screening: Rejects 80% of names like 'Jamal', accepts 60% of names like 'Brad', same qualifications",
                "is_unfair": True,
                "explanation": "Name-based discrimination indicates unfair racial bias in algorithm"
            },
            {
                "text": "Credit scoring: Lower scores for single mothers vs married couples with identical financial profiles",
                "is_unfair": True,
                "explanation": "Marital/family status creating different outcomes suggests unfair treatment"
            },
            {
                "text": "Voice assistant: Understands male voices 90% accuracy, female voices 70% accuracy",
                "is_unfair": True,
                "explanation": "Gender-based performance differences indicate unfair training bias"
            },
            {
                "text": "Facial recognition: 99% accuracy for ages 20-40, 75% accuracy for ages 65+",
                "is_unfair": True,
                "explanation": "Age-based accuracy disparities suggest unfair underrepresentation in training"
            },
            {
                "text": "Job matching: Suggests nursing jobs to women, engineering jobs to men with similar skills",
                "is_unfair": True,
                "explanation": "Gender stereotyping in job recommendations reflects unfair societal assumptions"
            },
            # FAIR scenarios
            {
                "text": "Medical AI: Higher diabetes risk scores for patients with family history vs those without",
                "is_unfair": False,
                "explanation": "Genetic factors are legitimate medical risk factors, making this fair treatment"
            },
            {
                "text": "Loan approval: Higher approval rates for borrowers with 800+ credit scores vs 600- credit scores",
                "is_unfair": False,
                "explanation": "Credit score is a legitimate factor for loan risk assessment, making this fair"
            },
            {
                "text": "Hiring algorithm: Prefers candidates with relevant work experience over those without",
                "is_unfair": False,
                "explanation": "Relevant experience is a legitimate job qualification, making this fair"
            },
            {
                "text": "Insurance pricing: Higher premiums for customers with multiple previous claims vs no claims",
                "is_unfair": False,
                "explanation": "Claims history is actuarially justified risk factor, making this fair"
            },
            {
                "text": "Academic admissions: Higher acceptance rates for students with better test scores and grades",
                "is_unfair": False,
                "explanation": "Academic performance metrics are relevant to educational outcomes, making this fair"
            },
            {
                "text": "Fraud detection: Flags unusual spending patterns as potential fraud",
                "is_unfair": False,
                "explanation": "Unusual patterns are legitimate fraud indicators, making this fair"
            },
            {
                "text": "Medical screening: Recommends mammograms for women over 50, not men",
                "is_unfair": False,
                "explanation": "Gender-specific medical recommendations based on biological differences are fair"
            }
        ]

        self.current_scenario = 0
        self.score = 0
        self.responses = []
        self.start_time = None
        self.response_times = []

        random.shuffle(self.scenarios)

        # Create widgets
        self.scenario_display = HTML(
            value="<h3>Fairness Pattern Recognition Training</h3>",
            layout=Layout(text_align='center')
        )

        self.unfair_button = Button(
            description='UNFAIR',
            button_style='danger',
            layout=Layout(width='200px', height='60px', margin='5px')
        )

        self.fair_button = Button(
            description='FAIR',
            button_style='success',
            layout=Layout(width='200px', height='60px', margin='5px')
        )

        self.next_button = Button(
            description='NEXT →',
            button_style='',
            layout=Layout(width='100px', height='30px', margin='10px')
        )

        self.feedback_display = HTML(layout=Layout(text_align='center'))
        self.progress_display = HTML(layout=Layout(text_align='center'))

        # Button clicks
        self.unfair_button.on_click(lambda b: self.handle_response(True))
        self.fair_button.on_click(lambda b: self.handle_response(False))
        self.next_button.on_click(lambda b: self.next_scenario())

        # Centered layout
        answer_buttons = HBox(
            [self.unfair_button, self.fair_button],
            layout=Layout(justify_content='center')
        )

        next_button_box = HBox(
            [self.next_button],
            layout=Layout(justify_content='center')
        )

        self.widget = VBox([
            self.scenario_display,
            answer_buttons,
            self.feedback_display,
            next_button_box,
            self.progress_display
        ])

    def start_training(self, num_scenarios=10):
        self.scenarios = self.scenarios[:num_scenarios]
        self.current_scenario = 0
        self.score = 0
        self.responses = []
        self.response_times = []
        self.show_scenario()
        return self.widget

    def show_scenario(self):
        if self.current_scenario >= len(self.scenarios):
            self.show_results()
            return

        scenario = self.scenarios[self.current_scenario]
        self.start_time = time.time()

        self.scenario_display.value = f"""
        <div style='border: 2px solid #ddd; padding: 20px; border-radius: 10px; background: #f9f9f9; text-align: center;'>
            <h3>Question {self.current_scenario + 1} of {len(self.scenarios)}</h3>
            <p style='font-size: 16px; line-height: 1.5;'><strong>{scenario['text']}</strong></p>
            <p style='color: #666;'><em>Is this FAIR or UNFAIR?</em></p>
        </div>
        """
        self.feedback_display.value = ""
        self.update_progress()

    def handle_response(self, user_says_unfair):
        if self.start_time is None:
            return

        response_time = time.time() - self.start_time
        scenario = self.scenarios[self.current_scenario]
        correct = user_says_unfair == scenario['is_unfair']

        if correct:
            self.score += 1

        self.responses.append({
            'correct': correct,
            'response_time': response_time
        })
        self.response_times.append(response_time)

        # Show feedback
        color = "green" if correct else "red"
        text = "✓ Correct!" if correct else "✗ Incorrect"

        self.feedback_display.value = f"""
        <div style='border: 2px solid {color}; padding: 15px; border-radius: 8px; margin: 10px 0; text-align: center;'>
            <h4 style='color: {color}; margin: 0;'>{text}</h4>
            <p><strong>Explanation:</strong> {scenario['explanation']}</p>
            <p><em>Response time: {response_time:.1f} seconds</em></p>
        </div>
        """

        self.start_time = None
        self.update_progress()

    def next_scenario(self):
        self.current_scenario += 1
        self.show_scenario()

    def update_progress(self):
        if len(self.responses) > 0:
            accuracy = self.score / len(self.responses) * 100
            avg_time = sum(self.response_times) / len(self.response_times)
        else:
            accuracy = avg_time = 0

        self.progress_display.value = f"""
        <div style='background: #e8f4fd; padding: 10px; border-radius: 5px; text-align: center;'>
            <strong>Progress:</strong> {len(self.responses)}/{len(self.scenarios)} |
            <strong>Accuracy:</strong> {accuracy:.1f}% |
            <strong>Avg Time:</strong> {avg_time:.1f}s
        </div>
        """

    def show_results(self):
        accuracy = self.score / len(self.scenarios) * 100
        avg_time = sum(self.response_times) / len(self.response_times)

        self.scenario_display.value = f"""
        <div style='border: 3px solid #4CAF50; padding: 25px; border-radius: 15px; background: #f8fff8; text-align: center;'>
            <h2 style='color: #4CAF50;'>Training Complete! 🎯</h2>
            <h3>Results:</h3>
            <ul style='list-style: none; padding: 0;'>
                <li><strong>Accuracy:</strong> {accuracy:.1f}% ({self.score}/{len(self.scenarios)})</li>
                <li><strong>Average Time:</strong> {avg_time:.1f} seconds</li>
            </ul>
        </div>
        """

        # Hide buttons
        self.unfair_button.layout.display = 'none'
        self.fair_button.layout.display = 'none'
        self.next_button.layout.display = 'none'
        self.feedback_display.value = ""

In [ ]:
#@title Start Simulator
# Start the fairness pattern recognition training
trainer = FairnessPatternTrainer()
widget = trainer.start_training(15)  # Practice with 15 scenarios
display(widget)